# EC5 - vibrations

In [ ]:
import math
import numpy as np

For residential floors with a fundamental frequency greater than 8 Hz ($f_1$ > 8 Hz) the following requirements should be satisfied:

$\frac{w}{F}\le a$, in $mm/kN$

$v\le b^{f_1\zeta-1} $, in $m/(Ns^2)$

where:
- $w$ is the maximum instantaneous vertical deflection caused by a vertical concentrated static force F applied at any point on the floor, taking account of load distribution;
- $v$ is the unit impulse velocity response, i.e. the maximum initial value of the vertical floor vibration velocity (in m/s) caused by an ideal unit impulse (1 Ns) applied at the point of the floor giving maximum response. Components above 40 Hz may be disregarded;
- $\zeta=0.01$ is the modal damping ratio.

The recommended range of limiting values of a and b and the recommended relationship between
a and b is given in the figure below.

![a-b relation](./figs/ec5-vibr-ab.png)


The calculations should be made under the assumption that the floor is unloaded, i.e., only the mass corresponding to the self-weight of the floor and other permanent actions.

For a rectangular floor with overall dimensions $l\times t$, simply supported along all four edges
and with timber beams having a span $l$, the fundamental frequency $f_1$ may approximately be calculated as

$f_1= \frac{\pi}{2l^2}\sqrt{\frac{(EI)_l}{m}}$

where:
- $m$ is the mass per unit area in $\text{kg/m}^2$;
- $l$ is the floor span, in m;
- $t$ is the floor width, in m;
- $(EI)_l$ is the equivalent plate bending stiffness of the floor about an axis perpendicular to the beam direction, in $\text{Nm}^2\text{/m}$.

The value $v$ may, as an approximation, be taken as:

$v=\frac{\left(1.6 + 2.4n_{40} \right)}{md+200}$

where:
- v is the unit impulse velocity response, in $m/(Ns^2)$
- $n_{40}$ is the number of first-order modes with natural frequencies up to 40 Hz;

The value of $n_{40}$ may be calculated from:

$n_{40}=\sqrt[4]{ {\left( {\left( \frac{40}{f_1} \right)}^2 - 1\right)} {\left( \frac{b}{l}\right)}^4 \frac{(EI)_l}{(EI)_t} }$

where $(EI)_t$ is the equivalent plate bending stiffness, in $Nm^2/m$, of the floor about an axis parallel
to the beams $\left( (EI)_t \lt (EI)_l \right)$.

In [ ]:
# useful functions
aval = [0.5, 1, 2, 4]
bval = [150, 120, 80, 50]

def floor_freq(l, EI, m):
    f1 = (math.pi/2/l**2)*math.sqrt(EI/m)
    return f1

def vel(f1, t, l, m, EI_l, EI_t):
    n40 = math.pow(((40/f1)**2-1.0)*((t/l)**4)*(EI_l/EI_t), 0.25)
    print(f"n40 = {n40}")
    v = 4*(0.4+0.6*n40)/(m*t*l+200)
    return v

def b_from_a(a):
    return np.interp(a, aval, bval)

def a_from_b(b):
    return np.interp(b, np.flip(bval), aval)

def vlim(f1, b, damp):
    return math.pow(b, f1*damp-1.0)

In [ ]:
# eurocode parameters
damp = 0.01
apar = 1.5 # (mm/kN)
bpar = b_from_a(apar)
print(f"a = {apar:.2f} mm/kN, b = {bpar:.1f}")

### Vibration verification in a rectangular floor

The floor is made by beams ($b\times h$) spanning through lengh L aparted by a distance $s$. The pavement over the beams has thickness $t$.

In [ ]:
# pavement definition
b = 0.045 # (m) beam size
h = 0.22 # (m) beam height
s = 0.4 # (m) beam spacing
tf = 0.018 # (m) floor thickness
El = 9000 # (MPa) beam modulus of elasticity
Et = 4930 # (MPa) floor modulus of elasticity
L = 3.7 # (m) pavement (beam) length
B = 4.4 # (m) pavement width
mass = 35 # (kg/m2)

In [ ]:
Il = b*h**3/12.0/s
It = tf**3/12.0
EI_l = El*Il*1e6
EI_t = Et*It*1e6
Il, It, round(EI_l,0), round(EI_t, 0)

In [ ]:
# Frequency calculation
f1 = floor_freq(L, EI_l, mass)
alfa = (B/L)*math.pow(EI_l/EI_t, 0.25)
fplate = f1*math.sqrt(1+1/alfa**4)
print(f"f1 = {f1:.2f} Hz, f1 > 8 Hz = {f1 > 8}, fplate = {fplate:.3f} Hz")

In [ ]:
# Calculation for velocity
v = vel(f1, B, L, mass, EI_l, EI_t)
vl = vlim(f1, bpar, damp)
print(f"v = {v:.4f} m/(Ns2), vlim = {vl:.4f} m/(Ns2), Check = {v < vl}")

In [ ]:
# Calculation for deflection
acalc = 1000*L**3/48/EI_l*1000
beta = EI_l/EI_t*(s/L)**4
kapa = 0.8+0.2*beta if beta > 0.3 else 0.4+2.9*beta-4.7*beta*beta
apar = a_from_b(bpar)
acalc *= kapa
print(f"w = {acalc:.3f} mm/kN, alim = {apar:.3f} mm/kN, Check = {acalc < apar}, k = {kapa:.3f}")